##### 问题:
我们想编写一个装饰器来包装函数，但是可以让用户调整装饰器的属性，这样在运行
时能够控制装饰器的行为。

##### 解决方案:
下面给出的解决方案对上一节的示例进行了扩展，引入了访问器函数（accessor
function），通过使用 nonlocal 关键字声明变量来修改装饰器内部的属性。之后把访问器
函数作为函数属性附加到包装函数上。

In [15]:
from functools import wraps, partial
import logging
# Utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        # Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
        return wrapper
    return decorate
# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y
    
@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')



DEBUG:__main__:add
DEBUG:__main__:Add called


5

下面的交互式会话展示了在完成上面的定义之后对各项属性的修改：

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2, 3)

In [ ]:
add.set_message('Add called')
add(2, 3)

In [ ]:
add.set_level(logging.WARNING)
add(2, 3)

本节示例的关键就在访问器函数（即，set_message()和 set_level()），它们以属性的形式
附加到了包装函数上。每个访问器函数允许对 nonlocal 变量赋值来调整内部参数。

这个示例中有一个令人惊叹的特性，那就是访问器函数可以跨越多个装饰器层进行传
播（如果所有的装饰器都使用了@functools.wraps 的话）。例如，假设引入了一个额外
的装饰器，比如 9.2 节中的@timethis，然后编写了如下的代码：

In [ ]:
import time
from functools import wraps
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1 

In [ ]:
countdown(10000000)

In [ ]:
countdown.set_level(logging.WARNING)
countdown.set_message("Counting down to zero")
countdown(10000000)

如果把装饰器的顺序像下面这样颠倒一下，就会发现访问器函数还是能够以相同的方
式工作。

In [ ]:
@logged(logging.DEBUG)
@timethis
def countdown(n):
    while n > 0:
        n -= 1

尽管这里没有给出，我们也可以通过添加如下额外的代码来实现用访问器函数返回内
部的状态值：


In [ ]:
'''
    @attach_wrapper(wrapper)
    def get_level():
        return level
    # Alternative
    wrapper.get_level = lambda: level
'''

本节中一个极为微妙的地方在于为什么要在一开始使用访问器函数。比方说，我们可
能会考虑其他的方案，完全基于对函数属性的直接访问，示例如下：

In [ ]:
'''
@wraps(func)
def wrapper(*args, **kwargs):
    wrapper.log.log(wrapper.level, wrapper.logmsg)
    return func(*args, **kwargs)
# Attach adjustable attributes
wrapper.level = level
wrapper.logmsg = logmsg
wrapper.log = log
'''

这种方法只能用在最顶层的装饰器上。如果在当前顶层的装饰器上又添加了一个装饰
器（比如示例中的@timethis），这样就会隐藏下层的属性使得它们无法被修改。而使用
访问器函数可以绕过这个限制。

最后但同样重要的是，本节展示的解决方案可以作为类装饰器的一种替代方式，我们
在 9.9 节中会继续谈到相关的主题。